
# Generate Readme with up to date list of xRunBooks

## Input

### Import libraries

In [ ]:
import os
import json
import requests
import urllib.parse
import copy
from pathlib import Path
import markdown
import nbformat
from nbconvert import MarkdownExporter
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)
try:
    from git import Repo
except:
    !pip install GitPython
    from git import Repo

### Variables

In [ ]:
# README variables
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"
badge_var = "[[BADGE]]"

# welcome variables
#this is a TODO
#welcome_template = "Welcome_template.ipynb"
#welcome = "Welcome.ipynb"
#replace_var_quote = f'"[[DYNAMIC_LIST]]",\n'


# Others
current_file = '.'
notebook_ext = '.ipynb'
github_url_base = 'https://github.com/unskript/Awesome-CloudOps-Automation/tree/master'
local_OSS_url = 'http://127.0.0.1:8888/lab/tree'
#fix these!
github_download_url = 'https://raw.githubusercontent.com/unskript/Awesome-CloudOps-Automation/master/'
unSkript_logo ='https://unskript.com/assets/favicon.png'


### Get files list

In [ ]:
repo = Repo('.')
branch =  repo.active_branch
list_of_dir = f"https://api.github.com/repos/unskript/Awesome-CloudOps-Automation/git/trees/{branch}?recursive=1"
r_gh = requests.get(list_of_dir).json().get("tree")
#print(branch)
notebooks = []
actions=[]
actionCount = 0
runBookCount = 0

main_readme_chart = ""
runbook_list=""
runbook_connector = ""
runbook_connector_list = []

action_list = ""
action_connector = ""
action_connector_list = []

connector_readme_connector =''
connector_readme = ''
hasRunbook = False
hasAction = False

if r_gh is not None:
    for file in r_gh:

        #only look at files we care about - ignore some of the directories that we don't need to scan
        if ".github" not in file.get("path") and ".gitignore" not in file.get("path") and "templates" not in file.get("path") and "/" in file.get("path")and "__init__" not in file.get("path") and "custom" not in file.get("path")and "unskript-ctl" not in file.get("path"):
            #runbooks are .ipynb, and actions are .py
            if file.get("path").endswith(".ipynb") or file.get("path").endswith(".py"):
                temp = file.get("path").split("/")
                if temp == -1:
                    data = {
                        "root": None,
                        "filename": file.get("path")
                    }
                    notebooks.append(data)
                    
                else:
                    isAction = False
                    last_folder = ""
                    file_name = temp[-1]
                    filename_string = file_name[0:file_name.find(".")]

                    temp.pop()
                    path = ""
                    for folder in temp:
                        path = path + folder +"/"
                        last_folder += "/" + folder
                    
                    # to be an action, the file must have the 2nd directory be lego, and there must be 3 layers of directory
                    if len(temp) ==3 and temp[1] =="legos":
                        isAction = True
                    #testing
                    
                    
                    
                    #JSON data
                    filename_json = Path(path +"/"+filename_string+ ".json")
                    jsonData = json.loads(filename_json.read_text())
                    
                    
                    ##we now have a path.. but only really need the root folder
                    ## different ways to generate fior action vs runbook
                    if isAction:
                        actionCount += 1
                        #this is an action folder
                        #find first slash
                        firstslash = last_folder.find("/",1)
                        root = last_folder[1:firstslash]
                        name = jsonData['action_title']
                        description = jsonData['action_description']
                        if 'action_categories' in jsonData:
                            categories = jsonData['action_categories']
                        else:
                            print(f"{name} is missing categories")
                        polling = jsonData['action_supports_poll']
                        iteration = jsonData['action_supports_iteration']
                        #not the python file - but the readme
                        github_url = f"{github_url_base}{last_folder}/README.md"   
                        
                        
                    else:
                        runBookCount+=1
                        #root folder for notebooks
                        root = last_folder[1:]
                        name = jsonData['name']
                        description = jsonData['description']
                        categories = jsonData['categories']
                        github_url = github_url_base+"/"+file.get("path")
                    
                    data = {
                        "root": root,
                        "filename": file_name,
                        "name": name,
                        "description": description,
                        "categories":categories,
                        "github_url": github_url
                    }
                    
                    if isAction:
                        data['type'] = "Action"
                        data['polling'] = polling
                        data['iteration']=iteration
                        actions.append(data)
                    else:
                        data['type'] = "RunBook"
                        local_url = local_OSS_url+"/"+file.get("path")
                        data['local_url'] = local_url  
                        notebooks.append(data)
                        
                        
                    #generate the list of runbooks for tha main readme
                    if not isAction:
                        main_readme_chart += f"|{root} |[{name}]({github_url}) | [Open in Browser]({local_url}) | \n"
    
                    #generate the runbook list page
                    if not isAction:
                        #have we created a category yet?
                        if runbook_connector == "":
                            runbook_connector = root
                            runbook_connector_list.append(runbook_connector)
                            runbook_list += f"\n# {runbook_connector}\n"
                        #same category, or new one
                        if runbook_connector != root:
                            # new category
                            runbook_connector = root
                            runbook_connector_list.append(runbook_connector)
                            runbook_list += f"\n# {runbook_connector}\n"
                            
                            
                        #now add in each runbook
                        runbook_list += f"* [{name}]({github_url}): {description}\n"
            
                    #generate the action list page
                    if isAction:
                        #have we created a category yet?
                        if action_connector == "":
                            action_connector = root
                            action_connector_list.append(action_connector)
                            action_list += f"\n# {action_connector}\n"
                        #same category, or new one
                        if action_connector != root:
                            # new category
                            action_connector = root
                            action_connector_list.append(action_connector)
                            action_list += f"\n# {action_connector}\n"
                            
                        #now add in each Action
                        action_list += f"* [{name}]({github_url}): {description}\n"
                    
                    

                    #generate the readme for each connector
                    #have we created a category yet?
                    if connector_readme_connector == "":
                        connector_readme_connector = root
                        connector_readme = ''
                        hasRunbook = False
                        hasAction = False
                    #same category, or new one
                    if connector_readme_connector != root:
                        # starting a new readme
                        #first let's save the old one:
                        #print(connector_readme)
                        readme_file = f"{connector_readme_connector}/README.md"
                        f  = open(readme_file, "w+")
                        f.write(connector_readme)
                        f.close()
                        #now start building the new readme
                        connector_readme_connector = root
                        connector_readme = ''
                        hasRunbook = False
                        hasAction = False

                    if data['type'] =="RunBook":
                        if not hasRunbook:
                            connector_readme += f'# {root} RunBooks\n'
                            hasRunbook = True
                        connector_readme += f"* [{name}]({github_url}): {description}\n"
                    if data['type'] =="Action":
                        if not hasAction:
                            connector_readme += f'\n# {root} Actions\n'
                            hasAction = True
                        connector_readme += f"* [{name}]({github_url}): {description}\n"
    
print(actionCount)
#print(action_list)
#print(action_connector_list)
#print(runbook_connector_list)
#print(runBookCount, actionCount)


In [ ]:
## generate category list and category url list for runbooks

notebook_categories = {}
notebook_category_urls = {}


for notebook in notebooks:
    #print(notebook)
    #print(notebook['root'], notebook['name'])
    if len(notebook['categories'])>0:
        #print(notebook['categories'])
        for category in notebook['categories']:
            if not category in notebook_categories:
                notebook_categories[category] = []
            notebook_categories[category].append(notebook)
            category_name = category[14:]
            notebook_category_urls[category_name] = Path(f"runbook_{category_name}.md")


In [ ]:
##headers for al of the runbook pages 
category_header1 ="|"
category_header2 ="|"
category_table ="|"

connector_header1 ="|"
connector_header2 ="|"
connector_table ="|"
counter =0
for connector in runbook_connector_list:
    counter+= 1
    if counter <3:
        connector_header1 += f" |"
        connector_header2 += f" ---|"
    connector_table += f" [{connector}](xRunBook_List.md#{connector}) |"      
    if counter%3 ==0:
        #start a new row of category
        connector_table += f"\n |"
        
connector_markdown_table = f"{connector_header1} \n {connector_header2} \n {connector_table} \n\n"

#print(notebook_category_urls)
counter =0
for categoryname in notebook_category_urls:
    counter+= 1
    if counter <3:
        category_header1 += f" |"
        category_header2 += f" ---|"
    category_table += f" [{categoryname}]({notebook_category_urls[categoryname]}) |"
    if counter%3 ==0:
        #start a new row of category
        category_table += f"\n |"    
category_markdown_table = f"{category_header1} \n {category_header2} \n {category_table} \n"
#this builds the Runbook list page
runbook_list = f"# RunBook Connectors:\n {connector_markdown_table} \n# RunBook Categories:\n {category_markdown_table} \n\n {runbook_list}"
       


In [ ]:
## generate category lists from actions

action_categories = {}
action_category_urls = {}


for action in actions:
    #print(action)
    if len(action['categories'])>0:
        #print(notebook['categories'])
        for category in action['categories']:
            if not category in action_categories:
                action_categories[category] = []
            action_categories[category].append(action)
            category_name = category[14:]
            action_category_urls[category_name] = Path(f"action_{category_name}.md")

In [ ]:
##headers for al of the Action pages 
category_header1 ="|"
category_header2 ="|"
category_table ="|"

action_header1 ="|"
action_header2 ="|"
action_table ="|"
gitbooklist = ""
counter =0
for action in action_connector_list:
    counter+= 1
    if counter <3:
        action_header1 += f" |"
        action_header2 += f" ---|"
    action_url = f"action_{action.upper()}.md"
    action_url= action_url.replace("KUBERNETES", "K8S")
    action_url = action_url.replace("MONGO", "MONGODB")
    action_url= action_url.replace("action_ELASTICSEARCH", "action_ES")
    action_table += f" [{action}]({action_url}) |"
    #print(action_url)
    gitbooklist += f"      * [{action}](action_url) \n"
    if counter%3 ==0:
        #start a new row of category
        action_table += f"\n |"

action_connector_markdown_table = f"{action_header1} \n {action_header2} \n {action_table} \n\n"
#print(action_connector_markdown_table)
counter = 0
for categoryname in action_category_urls:
    category_printed = categoryname
    counter+= 1
    if counter <3:
        category_header1 += f" |"
        category_header2 += f" ---|"
    category_url = str(action_category_urls[categoryname])
    category_url = category_url.replace("KUBERNETES", "K8S")
    category_url = category_url.replace("MONGO", "MONGODB")
    category_url = category_url.replace("POSTGRES", "POSTGRESQL")
    category_table += f" [{category_printed}]({category_url}) |"
    gitbooklist += f"      * [{category_printed}](lists/{action_category_urls[categoryname]})\n"
    if counter%3 ==0:
        #start a new row of category
        category_table += f"\n |"

action_category_markdown_table = f"{category_header1} \n {category_header2} \n {category_table} \n"


#print(action_connector_markdown_table)
action_list =f"# Actions By Connector:\n{action_connector_markdown_table} \n # Actions By Category: \n{action_category_markdown_table}\n\n\n\n "   
print(action_list)

In [ ]:

#generate pages for each Runbook category page
category_name =""
category_listing = ""
for category in notebook_categories:
    #category change - we need a new header
    if category_name =="":
        #new category
        category_name = category[14:]
        category_listing = f"# RunBook Connectors:\n {connector_markdown_table} \n# RunBook Categories:\n {category_markdown_table} \n# Runbooks in {category_name.replace('_', ' ')}\n"
    elif category_name != category:
        # we have finished off a category
        #save the oldcategory
        #print(category_name, category_listing)
        categoryList_filename = f"lists/{notebook_category_urls[category_name]}"
        f  = open(categoryList_filename, "w+")
        f.write(category_listing)
        f.close()
        category_name = category[14:]
        category_listing = f"# RunBook Connectors:\n {connector_markdown_table} \n# RunBook Categories:\n {category_markdown_table}\n # Runbooks in {category_name.replace('_', ' ')}\n"
    
    #print(notebook_categories[category])
    for runbook in notebook_categories[category]:
        category_listing += f"* {runbook['root']} [{runbook['name']}]({runbook['github_url']}): {runbook['description']}\n"
#finished loop -wrote last category
#print(category_listing)
f  = open(categoryList_filename, "w+")
f.write(category_listing)
f.close()


In [ ]:
# generate all of the action category pages 
category_name =""
category_listing = ""
for category in action_categories:
    if category_name =="":
        #new category
        
        category_name = category[14:]
        if category_name =="ELASTICSEARCH":
            category_name =="ES"
        elif category_name =="KUBERNETES":
            category_name =="K8S"
        
        elif category_name =="MONGO":
            print("MONGO")
            category_name =="MONGODB"
        category_listing = f"# Actions in the {category_name.replace('_', ' ')} category\n"
    elif category_name != category:
        # we have finished off a category
        #save the oldcategory
        #print(category_name, category_listing)
        #print(category)
        #print(category_listing)
        #place the links at the bottom
        category_listing += f"\n# Actions By Connector:\n{action_connector_markdown_table} \n # Actions By Category: \n{action_category_markdown_table} \n\n"
        categoryList_filename = f"lists/{action_category_urls[category_name]}"
        f  = open(categoryList_filename, "w+")
        f.write(category_listing)
        f.close()
        category_name = category[14:]
        if category_name =="ELASTICSEARCH":
            category_name =="ES"
        elif category_name =="KUBERNETES":
            category_name =="K8S"
        elif category_name =="MONGO":
            print("mongo")
            category_name =="MONGODB"
        category_listing = f"# Actions in the {category_name.replace('_', ' ')} category\n"
    
    #print(category_listing)
    for action in action_categories[category]:
        
        category_listing += f"* **{action['root']}**: [{action['name']}]({action['github_url']}): {action['description']}\n\n"
# last category is compelted when loop ends

category_listing += f"\n# Actions By Connector:\n{action_connector_markdown_table} \n # Actions By Category: \n{action_category_markdown_table} \n\n"
categoryList_filename = f"lists/{action_category_urls[category_name]}"
f  = open(categoryList_filename, "w+")
f.write(category_listing)
f.close()
        
        
#print(action_categories)
#print(action_category_urls)


### Preview the generated list

### Generate readme for github repository

In [ ]:
# Open README template
template = open(readme_template).read()

# Replace var to get list of templates in markdown format
template = template.replace(replace_var, main_readme_chart)

#create the action and runbook badges
actionImage = f'https://img.shields.io/static/v1?label=ActionCount&message={actionCount}&color=green'
runbookImage = f'https://img.shields.io/static/v1?label=xRunBookCount&message={runBookCount}&color=orange'
actionbadge = f"<img src={actionImage}>"
runbookbadge =  f"<img src={runbookImage}>"


#insert the badges on the readme
badgeCode = actionbadge + runbookbadge
template =  template.replace(badge_var, badgeCode)

# Save README
f  = open(readme, "w+")
f.write(template)
f.close()

In [ ]:

#generate a page of all the actions that are available
ac_list= readme = "lists/Action_list.md"
# Save README
f  = open(ac_list, "w+")
f.write(action_list)
f.close()


In [ ]:
#generate a page of all the Runbooks that are available
rb_list = readme = "lists/xRunBook_list.md"
# Save README
f  = open(rb_list, "w+")
f.write(runbook_list)
f.close()

In [ ]:
#create JSON files that can be used t build badges for the website showing our Runbook and action counts

import json
#store Runbookcount json (to create shield on webpage)
rb_list = readme = ".github/images/runbookShield.json"
runBookCount = str(runBookCount)
json1 = {"schemaVersion": 1,"label": "RunBook Count","message": runBookCount,"color": "orange"}

# Save README

with open(rb_list, "w") as outfile:
    json.dump(json1, outfile)


#store action count json (to create shield on webpage)
rb_list = readme = ".github/images/actionShield.json"
actionCount = str(actionCount)
json2 = {"schemaVersion": 1,"label": "Action Count","message": actionCount,"color": "green"}

# Save README
with open(rb_list, "w") as outfile:
    json.dump(json2, outfile)